In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings

In [17]:
# Obtener el directorio actual
path = os.getcwd()

# Concatenar el directorio actual con la ruta al directorio de los datos
path_data = os.path.join(path, "raw")
os.listdir(path_data)

['AVAMET', 'CLEAN_DATA', 'GVA', 'IVIA']

In [18]:
path_save = os.path.join(path_data, "CLEAN_DATA")

In [19]:
name_cols = ['Fecha', 'Hora', 'Velocidad', 'Direccion', 'Temperatura', 'Humedad']

Cada carpeta hace referencia a las diferentes fuentes de los datos climáticos. Puesto que cada formato tiene sus peculiaridades, vamos a limpiar los datos para obtener nuestros datos de forma única.

# 1. CLEANING GVA DATA

In [20]:
path_gva = os.path.join(path_data, "GVA")
os.listdir(path_gva)

['data_Alzira.txt', 'data_Artana.txt', 'data_Llutxent.txt', 'data_Montan.txt']

In [21]:
vars = ['FECHA', 'HORA', 'Veloc.', 'Direc.', 'Temp.', 'H.Rel.']

fechas = {
    'Alzira': ('2016-06-16', '2016-06-18'),
    'Artana': ('2016-07-25', '2016-07-27'),
    'Llutxent': ('2018-08-06', '2018-08-08'),
    'Montan': ('2015-07-07', '2015-07-10')
}

In [22]:
for file, key in zip(os.listdir(path_gva), fechas.keys()):
    print(file)
    # Leemos el fichero y seleccionamos las variables de interés
    df = pd.read_csv(os.path.join(path_gva, file), sep="\t", encoding='ISO-8859-1')
    df = df[vars]
    df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y')

    # Fijamos el intervalo de interés
    inicio, fin = fechas[key]
    df = df[(df['FECHA'] >= inicio) & (df['FECHA'] <= fin)]
    

    # Cambiamos el formato de las fechas y variables.
    df['HORA'] = df['HORA'].astype('int64').astype('str')
    df['HORA'] = pd.to_datetime(df['HORA'], format='%H').dt.strftime('%H:%M:%S')
    print(df.HORA)
    try:
        df['Veloc.'] = df['Veloc.'].astype('float')
        df['Temp.'] = df['Temp.'].astype('float')
    except:
        df['Veloc.'] = df['Veloc.'].str.replace(",", ".").astype('float')
        df['Temp.'] = df['Temp.'].str.replace(",", ".").astype('float')
    df['Direc.'] = df['Direc.'].astype('float')
    df['H.Rel.'] = df['H.Rel.'].astype('float') 
    df['Veloc.'] = df['Veloc.'] * 3.6 #  m/s -> km/h

    # Cambiamos el nombre de las variables
    df.columns = name_cols

    # Guardamos el fichero
    name = file.split(".")[0]
    df.to_csv(os.path.join(path_save, name), sep = ";", decimal = ".", index = False)

data_Alzira.txt
4009    00:00:00
4010    01:00:00
4011    02:00:00
4012    03:00:00
4013    04:00:00
          ...   
4076    19:00:00
4077    20:00:00
4078    21:00:00
4079    22:00:00
4080    23:00:00
Name: HORA, Length: 72, dtype: object
data_Artana.txt
4945    00:00:00
4946    01:00:00
4947    02:00:00
4948    03:00:00
4949    04:00:00
          ...   
5012    19:00:00
5013    20:00:00
5014    21:00:00
5015    22:00:00
5016    23:00:00
Name: HORA, Length: 72, dtype: object
data_Llutxent.txt
5098    00:00:00
5099    01:00:00
5100    02:00:00
5101    03:00:00
5102    04:00:00
          ...   
5165    19:00:00
5166    20:00:00
5167    21:00:00
5168    22:00:00
5169    23:00:00
Name: HORA, Length: 72, dtype: object
data_Montan.txt
4487    00:00:00
4488    01:00:00
4489    02:00:00
4490    03:00:00
4491    04:00:00
          ...   
4578    19:00:00
4579    20:00:00
4580    21:00:00
4581    22:00:00
4582    23:00:00
Name: HORA, Length: 96, dtype: object


In [23]:
df = pd.read_csv(os.path.join(path_save, "data_Alzira"), sep = ";", decimal = ".")

# 2. Cleaning AVAMET data

In [24]:
path_avamet = os.path.join(path_data, "AVAMET")
os.listdir(path_avamet)
df = pd.read_csv(path_avamet + '\\' + os.listdir(path_avamet)[-1])
df

,esta,data ini,vent gra_mit,vent vel_mit,vent vel_max,temp mit_mit,hrel mit_mit,pres mit_mit,prec tot_sum,prec tot
0,c30m102e07,8/7/20 0:00,292,"4,7","0,0","26,4","51,0","1016,0","0,00","0,00"
1,c30m102e07,8/7/20 0:05,296,"4,3","6,8","26,5","49,0","1016,0","0,00","0,00"
2,c30m102e07,8/7/20 0:20,267,"6,1","10,0","25,9","52,0","1016,0","0,00","0,00"
3,c30m102e07,8/7/20 0:25,274,"6,3","10,0","25,9","52,0","1016,0","0,00","0,00"
4,c30m102e07,8/7/20 0:30,268,"5,3","10,0","25,9","52,0","1016,0","0,00","0,00"
...,...,...,...,...,...,...,...,...,...,...
2863,c30m136e03,8/11/20 23:35,293,"12,2","74,5","25,2","67,0","1012,2","0,00","0,00"
2864,c30m136e03,8/11/20 23:40,293,"17,3","74,5","25,1","71,0","1012,4","0,00","0,00"
2865,c30m136e03,8/11/20 23:45,293,"6,1","74,5","24,3","73,0","1012,5","0,00","0,00"
2866,c30m136e03,8/11/20 23:50,293,"11,2","74,5","24,3","73,0","1012,5","0,00","0,00"


In [25]:
name_cols = ['Fecha', 'Hora', 'Direccion', 'Velocidad', 'Temperatura', 'Humedad']
vars = ['Fecha', 'Hora', 'vent gra_mit', 'vent vel_mit', "temp mit_mit", "hrel mit_mit"]
fechas = {
    'Beneixama': ('2019-07-15', '2019-07-15', 'c28m140e17'),
    'Beniarda': ('2020-08-28', '2020-08-29', 'c31m057e01'),
    'VallGallinera': ('2020-08-08', '2020-08-10', 'c30m102e07')
}

In [26]:
for file, key in zip(os.listdir(path_avamet), fechas.keys()):
    print(file)
    # Leemos el fichero y separamos fecha y hora. 
    df = pd.read_csv(os.path.join(path_avamet, file))
    df[['Fecha', 'Hora']] = df['data ini'].str.split(" ", expand = True)
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%m/%d/%y')
    df['Hora'] = df['Hora'].str.zfill(5)
    df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M').dt.time
    

    # Seleccionamos y creamos las variables de interés. Fijamos el intervalo de interés
    inicio, fin, esta = fechas[key]
    df = df[(df['Fecha'] >= inicio) & (df['Fecha'] <= fin) & (df['esta'] == esta)]
    df = df[vars]

    # Cambiamos el formato de las fechas y variables.
    try:
        df['vent vel_mit'] = df['vent vel_mit'].astype('float')
        df['temp mit_mit'] = df['temp mit_mit'].astype('float')
        df['hrel mit_mit'] = df['hrel mit_mit'].astype('float')
    except:
        df['vent vel_mit'] = df['vent vel_mit'].astype('str').str.replace(",", ".").astype('float')
        df['temp mit_mit'] = df['temp mit_mit'].astype('str').str.replace(",", ".").astype('float')
        df['hrel mit_mit'] = df['hrel mit_mit'].astype('str').str.replace(",", ".").astype('float')
    
    # Cambiamos el nombre de las variables y guradamos el fichero.
    df.columns = name_cols
    name = file.split(".")[0]
    df.to_csv(os.path.join(path_save, name), sep = ";", decimal = ".", index = False)

data_Beneixama.csv
data_Beniarda.csv
data_VallGallinera.csv


# 3. Cleaning IVIA data

In [27]:
import json

In [28]:
path_ivia = os.path.join(path_data, "IVIA")
os.listdir(path_ivia)[0]

'data_Artana.txt'

In [29]:
vars = ['fecha', 'hora', 'tmedia', 'hmedia', 'viento', 'direccion_num']
name_cols = ['Fecha', 'Hora', 'Temperatura', 'Humedad', 'Velocidad', 'Direccion']

In [30]:
for file in os.listdir(path_ivia):
    print(file)

    # Leemos el fichero y seleccionamos las variables de interés
    with open(os.path.join(path_ivia, file), 'r') as f:
        data = json.load(f)

    df = pd.json_normalize(data, 'data')
    df = df[vars]

    # Cambiamos el formato de las fechas y nombre de las columnas.
    df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')

    # Convertir la columna 'Hora' a formato datetime
    df['hora'] = pd.to_datetime(df['hora'], format='%H:%M', errors='coerce')

    # Aplicar una corrección a los valores de hora que son mayores o iguales a 24
    df['hora'] = pd.to_datetime(df['hora'].mask(df['hora'].dt.hour >= 24, df['hora'].dt.strftime('00:%M')), format='%H:%M', errors='coerce').dt.strftime('%H:%M:%S')

    # Guardamos el fichero
    df.columns = name_cols
    name = file.split(".")[0]
    df.to_csv(os.path.join(path_save, name), sep = ";", decimal = ".", index = False)    

data_Artana.txt
data_Bolulla.txt
data_Carcaixent.txt
data_Chella.txt
data_Gatova.txt
data_Llutxent.txt
data_ValldEbo.txt
